#### 해외지수 수집하기 (JSON 파일 크롤링)
  + 네이버금융 기준 국내증시와 해외증시 페이지 구조가 다름
  + 자바스크립트: 웹사이트 전체 틀은 유지하면서 안 보이는 구석에서 데이터를 불러오고 일부 데이터를 바꿔주는 기법을 구사할 때 사용
    - JSON
  + 검사 -> Network -> 호출되는 주소 Copy->Copy link address

+ 구조: https://finance.naver.com/world/worldDayListJson.nhn?symbol=SPI@SPX&fdtc=0&page=1
+ symbol 부분과 page 부분

In [1]:
import pandas as pd
from urllib.request import urlopen
import json

In [2]:
symbol = 'SPI@SPX' # S&P500
page = 1

In [3]:
url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol=' \
+ symbol + '&fdtc=0&page=' + str(page)
raw = urlopen(url)
data = json.load(raw)

In [4]:
data[0] # symb: 종목 / xymd: 날짜 / clos: 종가

{'symb': 'SPI@SPX',
 'xymd': '20201217',
 'open': 3713.65,
 'high': 3725.12,
 'low': 3710.87,
 'clos': 3722.48,
 'diff': 21.31,
 'rate': 0.58,
 'gvol': 2195077600}

In [5]:
data[0]['symb']

'SPI@SPX'

In [6]:
data[0]['xymd']

'20201217'

In [7]:
data[0]['clos']

3722.48

In [8]:
d = dict() # 빈 딕셔너리
for n in range(len(data)):
    date = pd.to_datetime(data[n]['xymd']).date()
    price = float(data[n]['clos'])
    d[date] = price # {날짜: 종가} 형태로 딕셔너리에 저장
print(d)

{datetime.date(2020, 12, 17): 3722.48, datetime.date(2020, 12, 16): 3701.17, datetime.date(2020, 12, 15): 3694.62, datetime.date(2020, 12, 14): 3647.49, datetime.date(2020, 12, 11): 3663.46, datetime.date(2020, 12, 10): 3668.1, datetime.date(2020, 12, 9): 3672.82, datetime.date(2020, 12, 8): 3702.25, datetime.date(2020, 12, 7): 3691.96, datetime.date(2020, 12, 4): 3699.12}


In [9]:
# 함수화

def read_json(d, symbol, page=1): 
    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol=' \
    + symbol + '&fdtc=0&page=' + str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    for n in range(len(data)):
        date = pd.to_datetime(data[n]['xymd']).date()
        price = float(data[n]['clos'])
        d[date] = price
        
    if len(data) == 10:  # 다음 페이지를 불러올지 말지 결정    
        page += 1
        read_json(d, symbol, page)
        
    return(d)

In [10]:
historical_index = pd.Series()
historical_index = read_json(historical_index, symbol, page)

<ipython-input-10-a5426966decc>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  historical_index = pd.Series()


In [11]:
historical_index.head(3)

2020-12-17    3722.48
2020-12-16    3701.17
2020-12-15    3694.62
dtype: float64

In [12]:
historical_index.tail(3)

2002-01-04    1172.51
2002-01-03    1165.27
2002-01-02    1154.67
dtype: float64

In [13]:
# 여러 종목 수집
indices = {
    'SPI@SPX': 'S&P500',
    'NAS@NDX': 'NASDAQ100',
    'NII@NI225': 'NIKKEI225'
}

In [14]:
historical_indices = dict()
for key, value in indices.items():
    print(key, value)
    s = dict()
    s = read_json(s, key, 1)
    historical_indices[value] = s
prices_df = pd.DataFrame(historical_indices)

SPI@SPX S&P500
NAS@NDX NASDAQ100
NII@NI225 NIKKEI225


In [15]:
prices_df.head(3)

,S&P500,NASDAQ100,NIKKEI225
2020-12-17,3722.48,12752.06,26806.67
2020-12-16,3701.17,12668.16,26757.40
2020-12-15,3694.62,12595.92,26687.84


In [16]:
# 날짜 지정 --> 날짜가 지정되지 않으면 오늘 날짜를 기준으로 데이터를 수집하도록 처리하기 위함
def date_format(d=''):
    if d != '':
        this_date = pd.to_datetime(d).date()
    else:
        this_date = pd.Timestamp.today().date()
    return (this_date)

In [17]:
# 날짜 지정까지 추가한 전체 함수
def index_global(d, symbol, start_date='', end_date='', page=1):

    end_date = date_format(end_date)
    if start_date == '':
        start_date = end_date - pd.DateOffset(months=1)
    start_date = date_format(start_date)
        
    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol=' \
    + symbol + '&fdtc=0&page=' + str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    if len(data) > 0:
        
        for n in range(len(data)):
            date = pd.to_datetime(data[n]['xymd']).date()

            if date <= end_date and date >= start_date:   
                
                price = float(data[n]['clos'])
                d[date] = price
                
            elif date < start_date:   
                
                return (d)              

        if len(data) == 10:
            
            page += 1
            index_global(d, symbol, start_date, end_date, page)
        
    return (d)

In [18]:
# 실행
historical_indices = dict()
start_date = '2020-12-01'
end_date = '2020-12-15'
for key, value in indices.items(): 
    s = dict()
    s = index_global(s, key, start_date, end_date)
    historical_indices[value] = s    
prices_df = pd.DataFrame(historical_indices)

In [19]:
prices_df

,S&P500,NASDAQ100,NIKKEI225
2020-12-15,3694.62,12595.92,26687.84
2020-12-14,3647.49,12462.21,26732.44
2020-12-11,3663.46,12375.41,26652.52
2020-12-10,3668.10,12401.74,26756.24
2020-12-09,3672.82,12364.64,26817.94
2020-12-08,3702.25,12635.72,26467.08
2020-12-07,3691.96,12596.47,26547.44
2020-12-04,3699.12,12528.48,NaN
2020-12-03,3666.72,12467.13,NaN
2020-12-02,3669.01,12456.41,NaN


#### 발생하는 문제점
  + S&P500과 NASDAQ100은 정상적으로 수집이 됨
  + 하지만 NIKKEI225는 1페이지 이후로 넘어가면 수집이 안됨
    - 해결방안 찾는 중...